In [131]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_recall_curve, accuracy_score, precision_score, recall_score, f1_score, roc_curve

In [132]:
matches = pd.read_csv("Football.csv")
matches.head()

/Users/ufukkose/Desktop/projects/soccer-predictor/venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (4,5,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,71,72) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Country,League,home_team,away_team,home_score,away_score,season_year,Date_day,Date_hour,first_half,...,Distance_Covered_(km)_Home,Distance_Covered_(km)_Host,Clearances_Completed_Home,Clearances_Completed_Host,Pass_Success_per_Home,Pass_Success_per_Host,referee,venue,capacity,attendance
0,Germany,Bundesliga,B. Monchengladbach,Werder Bremen,4,1,2024/2025,3.11,19:30,3 - 0,...,NaN,NaN,NaN,NaN,NaN,NaN,Dankert B. (Ger),Borussia-Park (Mönchengladbach),54 042,54 042
1,Germany,Bundesliga,Freiburg,Mainz,0,0,2024/2025,3.11,17:30,0 - 0,...,NaN,NaN,NaN,NaN,NaN,NaN,Brand B. (Ger),Europa-Park Stadion (Freiburg),34 700,34 400
2,Germany,Bundesliga,Dortmund,RB Leipzig,2,1,2024/2025,2.11,20:30,1 - 1,...,NaN,NaN,NaN,NaN,NaN,NaN,Stieler T. (Ger),Signal Iduna Park (Dortmund),81 365,81 365
3,Germany,Bundesliga,Bayern Munich,Union Berlin,3,0,2024/2025,2.11,17:30,2 - 0,...,NaN,NaN,NaN,NaN,NaN,NaN,Jollenbeck M. (Ger),Allianz Arena (Munich),75 024,75 000
4,Germany,Bundesliga,Eintracht Frankfurt,Bochum,7,2,2024/2025,2.11,17:30,4 - 1,...,NaN,NaN,NaN,NaN,NaN,NaN,Schlager D. (Ger),Deutsche Bank Park (Frankfurt),58 000,58 000


In [133]:
matches.shape

(95384, 91)

In [134]:
db = matches[(matches["League"] == "Bundesliga") & 
             (matches["season_year"].isin(["2023/2024", "2022/2023"]))].copy()
db.shape

(616, 91)

In [135]:
db["home_team"] = db["home_team"].str.replace(r"\n2", "", regex=True).str.strip()
db["away_team"] = db["away_team"].str.replace(r"\n2", "", regex=True).str.strip()

In [136]:
print(db["home_team"].value_counts())
print(db["away_team"].value_counts())

Stuttgart              35
Bochum                 35
Freiburg               34
Eintracht Frankfurt    34
Werder Bremen          34
Dortmund               34
Bayern Munich          34
B. Monchengladbach     34
RB Leipzig             34
Wolfsburg              34
Hoffenheim             34
Mainz                  34
Union Berlin           34
Augsburg               34
Bayer Leverkusen       34
FC Koln                34
Hertha Berlin          17
Darmstadt              17
Schalke                17
Heidenheim             17
Hamburger SV            1
Dusseldorf              1
Name: home_team, dtype: int64
Bochum                 35
Stuttgart              35
RB Leipzig             34
Eintracht Frankfurt    34
Werder Bremen          34
Hoffenheim             34
Dortmund               34
Bayern Munich          34
B. Monchengladbach     34
Union Berlin           34
Wolfsburg              34
Mainz                  34
Freiburg               34
Augsburg               34
Bayer Leverkusen       34
FC Koln 

In [137]:
db = db[~db["home_team"].isin(["Hamburger SV", "Dusseldorf"]) &
        ~db["away_team"].isin(["Hamburger SV", "Dusseldorf"])]
print(db["home_team"].value_counts())
print(db["away_team"].value_counts())

Freiburg               34
Eintracht Frankfurt    34
Werder Bremen          34
Dortmund               34
Bayern Munich          34
B. Monchengladbach     34
RB Leipzig             34
Wolfsburg              34
Stuttgart              34
Hoffenheim             34
Mainz                  34
Union Berlin           34
Bochum                 34
Augsburg               34
Bayer Leverkusen       34
FC Koln                34
Hertha Berlin          17
Darmstadt              17
Schalke                17
Heidenheim             17
Name: home_team, dtype: int64
RB Leipzig             34
Eintracht Frankfurt    34
Werder Bremen          34
Hoffenheim             34
Dortmund               34
Bayern Munich          34
B. Monchengladbach     34
Union Berlin           34
Wolfsburg              34
Stuttgart              34
Mainz                  34
Freiburg               34
Bochum                 34
Augsburg               34
Bayer Leverkusen       34
FC Koln                34
Heidenheim             17
Hertha B

In [138]:
nunique = db.nunique()
constant_cols = nunique[nunique == 1].index.tolist()
print("Constant columns:", constant_cols)
db = db.drop(columns=constant_cols)

Constant columns: ['Country', 'League', 'Tackles_Home', 'Tackles_Host']


In [139]:
null_ratio = db.isnull().mean()
high_null_cols = null_ratio[null_ratio > 0.15].index.tolist()
print("Columns with too many null values:", high_null_cols)
db = db.drop(columns=high_null_cols)

Columns with too many null values: ['home_team_goals_current_time', 'home_team_goals_current_score', 'home_team_goals', 'home_team_goals_assist', 'away_team_goals_current_time', 'away_team_goals_current_score', 'away_team_goals', 'away_team_goals_assist', 'home_team_red_card_current_time', 'home_team_red_card', 'home_team_red_card_why', 'away_team_red_card_current_time', 'away_team_red_card', 'away_team_red_card_why', 'expected_goals_xg_home', 'expected_goals_xg_host', 'Blocked_Shots_Home', 'Blocked_Shots_Host', 'Red_Cards_Home', 'Red_Cards_Host', 'Crosses_Completed_Home', 'Crosses_Completed_Host', 'Interceptions_Home', 'Interceptions_Host', 'Distance_Covered_(km)_Home', 'Distance_Covered_(km)_Host', 'Clearances_Completed_Home', 'Clearances_Completed_Host', 'Pass_Success_per_Home', 'Pass_Success_per_Host']


In [140]:
db.shape

(612, 57)

In [141]:
null_columns = db.isnull().sum()
null_columns = null_columns[null_columns > 0]
print(null_columns)

first_half                               1
second_half                              1
home_team_yellow_card_current_time      74
home_team_yellow_card                   74
home_team_yellow_card_why               75
away_team_yellow_card_current_time      58
away_team_yellow_card                   58
away_team_yellow_card_why               59
home_team_substitutions_current_time     1
home_team_substitutions                  1
home_team_substitutions_with             1
home_team_substitution_why               1
away_team_substitutions_current_time     1
away_team_substitutions                  1
away_team_substitutions_with             1
away_team_substitution_why               1
Ball_Possession_Home                     1
Ball_Possession_Host                     1
Goal_Attempts_Home                       1
Goal_Attempts_Host                       1
Shots_on_Goal_Home                       1
Shots_on_Goal_Host                       1
Shots_off_Goal_Home                      1
Shots_off_G

In [142]:
null_vals = db.isnull().sum()
print("There are " + str(null_vals.sum()) + " missing values.")

There are 522 missing values.


In [143]:
db.columns

Index(['home_team', 'away_team', 'home_score', 'away_score', 'season_year',
       'Date_day', 'Date_hour', 'first_half', 'second_half',
       'home_team_yellow_card_current_time', 'home_team_yellow_card',
       'home_team_yellow_card_why', 'away_team_yellow_card_current_time',
       'away_team_yellow_card', 'away_team_yellow_card_why',
       'home_team_substitutions_current_time', 'home_team_substitutions',
       'home_team_substitutions_with', 'home_team_substitution_why',
       'away_team_substitutions_current_time', 'away_team_substitutions',
       'away_team_substitutions_with', 'away_team_substitution_why',
       'Ball_Possession_Home', 'Ball_Possession_Host', 'Goal_Attempts_Home',
       'Goal_Attempts_Host', 'Shots_on_Goal_Home', 'Shots_on_Goal_Host',
       'Shots_off_Goal_Home', 'Shots_off_Goal_Host', 'Free_Kicks_Home',
       'Free_Kicks_Host', 'Corner_Kicks_Home', 'Corner_Kicks_Host',
       'Offsides_Home', 'Offsides_Host', 'Throw_ins_Home', 'Throw_ins_Host',
     

In [144]:
db = db.drop(columns = ['home_team_yellow_card_current_time', 'home_team_yellow_card',
       'home_team_yellow_card_why', 'away_team_yellow_card_current_time',
       'away_team_yellow_card', 'away_team_yellow_card_why',
       'home_team_substitutions_current_time', 'home_team_substitutions',
       'home_team_substitutions_with', 'home_team_substitution_why',
       'away_team_substitutions_current_time', 'away_team_substitutions',
       'away_team_substitutions_with', 'away_team_substitution_why',])

In [145]:
db.shape

(612, 43)

In [146]:
numerical_cols = db.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_cols = db.select_dtypes(include=['object', 'category', 'bool']).columns.tolist()

print("Numerical columns:", numerical_cols)
print("Categorical columns:", categorical_cols)

Numerical columns: ['Date_day', 'Goal_Attempts_Home', 'Goal_Attempts_Host', 'Shots_on_Goal_Home', 'Shots_on_Goal_Host', 'Shots_off_Goal_Home', 'Shots_off_Goal_Host', 'Free_Kicks_Home', 'Free_Kicks_Host', 'Corner_Kicks_Home', 'Corner_Kicks_Host', 'Offsides_Home', 'Offsides_Host', 'Throw_ins_Home', 'Throw_ins_Host', 'Goalkeeper_Saves_Home', 'Goalkeeper_Saves_Host', 'Fouls_Home', 'Fouls_Host', 'Yellow_Cards_Home', 'Yellow_Cards_Host', 'Total_Passes_Home', 'Total_Passes_Host', 'Completed_Passes_Home', 'Completed_Passes_Host', 'Attacks_Home', 'Attacks_Host', 'Dangerous_Attacks_Home', 'Dangerous_Attacks_Host']
Categorical columns: ['home_team', 'away_team', 'home_score', 'away_score', 'season_year', 'Date_hour', 'first_half', 'second_half', 'Ball_Possession_Home', 'Ball_Possession_Host', 'referee', 'venue', 'capacity', 'attendance']


In [147]:
null_counts_per_row = db.isnull().sum(axis=1)
row_with_most_nulls = null_counts_per_row.idxmax()
null_ratio = db.isnull().sum(axis=1).div(len(db.columns))
print(f"Null ratio in worst row: {null_ratio[row_with_most_nulls]:.2%}")
db = db.drop(index=row_with_most_nulls)

Null ratio in worst row: 69.77%


In [148]:
null_columns = db.isnull().sum()
null_columns = null_columns[null_columns > 0]
print(null_columns)

first_half            1
second_half           1
Free_Kicks_Home      11
Free_Kicks_Host      11
Throw_ins_Home       13
Throw_ins_Host       13
Yellow_Cards_Home    18
Yellow_Cards_Host    18
dtype: int64


In [149]:
db.loc[db["first_half"].isnull(), "first_half"] = "0 - 0"
db.loc[db["second_half"].isnull(), "second_half"] = "1 - 0"
print(db.loc[(db['home_team'] == 'FC Koln') & (db['away_team'] == 'Freiburg')])

    home_team away_team home_score away_score season_year  Date_day Date_hour  \
104   FC Koln  Freiburg          0          0   2023/2024      4.05     19:30   
431   FC Koln  Freiburg          0          1   2022/2023     29.04     16:30   

    first_half second_half Ball_Possession_Home  ... Completed_Passes_Home  \
104      0 - 0       0 - 0                  55%  ...                 367.0   
431      0 - 0       1 - 0                  57%  ...                 350.0   

     Completed_Passes_Host  Attacks_Home  Attacks_Host  \
104                  289.0         133.0          90.0   
431                  241.0         102.0         103.0   

     Dangerous_Attacks_Home  Dangerous_Attacks_Host            referee  \
104                    61.0                    33.0   Storks S.  (Ger)   
431                    69.0                    42.0  Reichel T.  (Ger)   

                              venue  capacity  attendance  
104  RheinEnergieStadion  (Cologne)    50 000      50 000  
431